In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, log_loss, classification_report)
from imblearn.over_sampling import SMOTE
import xgboost


In [56]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()
ttrain=train.drop('Attrition',axis=1)
all_data = pd.concat((ttrain.loc[:,'Age':],
                      test.loc[:,'Age':]))
all_data = pd.get_dummies(all_data)
all_data.drop('EmployeeNumber',axis=1,inplace=True)
#all_data.drop(['Behaviour','EmployeeNumber','OverTime_Yes','MaritalStatus_Single','JobRole_Sales Representative','Gender_Male','BusinessTravel_Travel_Rarely','EducationField_Medical','Department_Human Resources'],axis=1,inplace=True)
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.Attrition

In [57]:
X_train

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,30,2,3,3,3,4,2564,0,14,3,...,0,0,0,0,0,0,0,1,1,0
1,36,12,4,3,3,3,4663,9,12,3,...,1,0,0,0,0,0,1,0,0,1
2,55,2,1,3,3,4,5160,4,16,3,...,0,0,0,1,0,0,0,1,1,0
3,39,24,1,1,3,4,4108,7,13,3,...,0,0,1,0,0,0,0,1,1,0
4,37,3,3,3,3,3,9434,1,15,3,...,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,42,19,3,3,4,3,2759,6,12,3,...,0,0,1,0,0,1,0,0,0,1
1624,55,2,1,3,3,4,5160,4,16,3,...,0,0,0,1,0,0,0,1,1,0
1625,25,9,2,1,2,1,4400,3,12,3,...,0,0,0,0,1,0,1,0,1,0
1626,29,13,3,1,2,1,2335,4,15,3,...,0,0,0,0,0,1,0,0,0,1


In [58]:
seed = 0   # We set our random seed to zero for reproducibility
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
#     'warm_start': True, 
    'max_features': 0.3,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}


In [59]:
rf = RandomForestClassifier(**rf_params)
rf.fit(X_train,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [37]:
rf_predictions = rf.predict_proba(X_test)[:,1]

In [38]:
# Scatter plot 
trace = go.Scatter(
    y = rf.feature_importances_,
    x = X_train.columns.values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = X_train.columns.values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [39]:
# Gradient Boosting Parameters
gb_params ={
    'n_estimators': 1500,
    'max_features': 0.9,
    'learning_rate' : 0.25,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'subsample': 1,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}
gb = GradientBoostingClassifier(**gb_params)
# Fit the model to our SMOTEd train and target
gb.fit(X_train, y)
# Get our predictions
gb_predictions = gb.predict_proba(X_test)[:,1]
print("Predictions have finished")

Predictions have finished


In [40]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label = y)
dtest = xgb.DMatrix(X_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

In [41]:
model_xgb = xgb.XGBClassifier(n_estimators=360, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv  n_estimators=360, max_depth=2, learning_rate=0.1
model_xgb.fit(X_train, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=360, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [42]:
# xgb_preds = np.expm1(model_xgb.predict_proba(X_test)[:,1])
# preds = 0.7*gb_predictions + 0.3*xgb_preds


In [43]:
xgb_preds = np.expm1(model_xgb.predict_proba(X_test)[:,1])
preds = 0.7*rf_predictions  + 0.3*xgb_preds


In [44]:
solution = pd.DataFrame({"id":test.Id, "Attrition":gb_predictions})
solution.to_csv("attritionxgb3.csv", index = False)